In [59]:
results_folder = "../results"

In [60]:
import os
import numpy as np

def get_logits(results_folder):
    all_logits = {}
    for root, dirs, files in os.walk(results_folder):
        
        if root == results_folder:
            continue
        print(root)
        model = root.split("/")[-1]
        logits = np.load(os.path.join(root, "logits.npz"))
        all_logits[model] = logits['arr_0']
        
    return all_logits

In [64]:
all_logits = get_logits(results_folder)
print(all_logits.keys())

# ### test
# all_logits['resnet18_noagument_2'] = all_logits['resnet18_noagument']
# print(all_logits.keys())

../results/resnet34
../results/vgg19
../results/resnet18
../results/resnet50
../results/vgg16
dict_keys(['resnet34', 'vgg19', 'resnet18', 'resnet50', 'vgg16'])


In [70]:
import json

manifest = "../data/manifest_full.json"
images_folder = "../data/wb_recognition_dataset/val/images"

def get_filenames(images_folder):
    return sorted(os.listdir(images_folder))

def get_decodevocab(manifest):
    with open(manifest, "r") as file:
        samples = json.load(file)["train"]
    
    keys = list(samples.keys())
    return dict(zip(range(len(keys)), keys))

def decode_labels(preds, decode_vocab):
    labels = []
    # print(decode_vocab)
    for pred in preds:
        pred = int(pred)
        label = decode_vocab[pred]
        labels.append(label)
    return labels

def soft_ensemble(all_logits, weights):
    assert sum(weights) == 1
    ensemble_logits = np.zeros(shape=list(all_logits.values())[0].shape)
    for i, (model, logits) in enumerate(all_logits.items()):
        ensemble_logits += logits * weights[i]
    
    ensemble_preds = np.argmax(ensemble_logits, axis=1)
    decode_vocab = get_decodevocab(manifest)
    ensemble_labels = decode_labels(ensemble_preds, decode_vocab)
    
    filenames = get_filenames(images_folder)
    results = {}
    for filename, ensemble_label in zip(filenames, ensemble_labels):
        results[filename.split('.')[0]] = int(ensemble_label)
    
    return ensemble_logits, results

In [71]:
ensemble_logits, results = soft_ensemble(all_logits, weights=[0.2, 0.2, 0.2, 0.2, 0.2])
ensemble_logits

array([[-36.8645196 , -34.357342  , -36.16119647, ..., -51.13641882,
        -55.58463097, -41.32209158],
       [-39.75089383, -39.77310944, -41.93241024, ..., -55.00753069,
        -55.37384462, -48.56921959],
       [-40.22456503, -41.92690444, -30.50220561, ..., -47.89273977,
        -59.77570152, -52.05369043],
       ...,
       [-31.12986851, -22.90482712, -31.55321503, ..., -47.50384283,
        -43.75393009, -41.64422941],
       [-39.24476814, -35.23463821, -41.24069834, ..., -56.94106054,
        -68.84997272, -66.80036974],
       [-16.90103889, -12.88231838, -25.65301132, ..., -47.12804604,
        -41.65291071, -44.45369101]])

In [73]:
import csv
def save_csv(result_preds, csv_path):
    result_preds = {**{"image_name":"label"}, **result_preds}
    with open(csv_path, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        
        for filename, class_id in result_preds.items():
            writer.writerow([filename, class_id])

In [74]:
save_csv(results, "../results/ensemble.csv")